In [3]:
import numpy as np
import anndata
import pandas as pd
import scanpy as sc
import scipy
import seaborn as sns

from moscot.problems.time._lineage import TemporalProblem

/home/mgander/miniconda3/envs/mc/lib/python3.10/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/home/mgander/miniconda3/envs/mc/lib/python3.10/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [4]:
Path="/home/mgander/Unrelated/moscotTime_Reproducibility/Data"
ts=['E3.5', 'E4.5', 'E5.25', 'E5.5', 'E6.25', 'E6.5', 'E6.75', 'E7.0', 'E7.25', 'E7.5', 'E7.75', 'E8.0', 'E8.25', 'E8.5a', 'E8.5b', 'E9.5', 'E10.5', 'E11.5', 'E12.5', 'E13.5']

cells=[32, 80, 100, 120, 400, 660, 1720, 4500, 8200, 15000, 30000, 60000, 73000, 90000, 90000, 200000, 1100000, 2600000, 6000000, 13000000]
Cell_number_dict={}
for i in range(20):
    Cell_number_dict[ts[i]]=cells[i]
    
# Doesn't matter which adata you use to create a tp instance, the result is always the same. I just need one
# to instantiate a tp-problem
adata=sc.read(f"{Path}/anndatas/Comb_anndatas/adata_E3.5_E4.5.h5ad")
tp=TemporalProblem(adata)

# Moscot maps to growth rates

In [25]:
for i in range(13):
    ts0=ts[i]
    ts1=ts[i+1]
    print(ts0)
    
    tp=tp.load(f'{Path}/moscot_maps/{ts0}_{ts1}_TemporalProblem.pkl')
    time_tuple=list(tp.solutions.keys())[0]
    gr=tp[time_tuple].solution.a
    gr=gr/gr.mean()*Cell_number_dict[ts1]/Cell_number_dict[ts0]
    np.save(f'{Path}/moscot_maps/{ts0}_growth_rates.npy', gr)

E3.5
E4.5
E5.25
E5.5
E6.25
E6.5
E6.75
E7.0
E7.25
E7.5
E7.75
E8.0
E8.25


# For E8.0--> E8.25 I need prior growth rates and map without growth rate intitialitions

In [15]:
# Prior growth rates and growth rates without initialization
i=11
ts0=ts[i]
ts1=ts[i+1]
print(ts0)

tp=tp.load(f'{Path}/moscot_maps/{ts0}_{ts1}_TemporalProblem.pkl')
adata=tp.adata
time_tuple=list(tp.solutions.keys())[0]
gr=np.array(list(tp.prior_growth_rates['prior_growth_rates'])[:len(adata[adata.obs['day']==time_tuple[0]])])
gr=gr/gr.mean()
np.save(f'{Path}/moscot_maps/{ts0}_init_growth_rates.npy', gr)

tp=tp.load(f'{Path}/moscot_maps/{ts0}_{ts1}_no_gr_init_TemporalProblem.pkl')
adata=tp.adata
gr=tp[time_tuple].solution.a
gr=gr/gr.mean()*Cell_number_dict[ts1]/Cell_number_dict[ts0]
np.save(f'{Path}/moscot_maps/{ts0}_growth_rates_no_gr_init.npy', gr)

E8.0


In [14]:
i=11
ts0=ts[i]
ts1=ts[i+1]
print(ts0)

tp=tp.load(f'{Path}/moscot_maps/{ts0}_{ts1}_TemporalProblem.pkl')
time_tuple=list(tp.solutions.keys())[0]
gr=tp[time_tuple].solution.a
gr=gr/gr.mean()*Cell_number_dict[ts1]/Cell_number_dict[ts0]
np.save(f'{Path}/moscot_maps/{ts0}_growth_rates_no_gr_init.npy', gr)

E8.0
